In [47]:
import pandas
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge

PATH = '/Users/vlasnikita/Documents/vls/py/ml_py_coursera/'

# Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv.
data_train = pandas.read_csv(PATH + 'lesson_4/salary-train.csv')
data_test = pandas.read_csv(PATH + 'lesson_4/salary-test-mini.csv')

# Приведите тексты к нижнему регистру
data_train['FullDescription'] = data_train['FullDescription'].str.lower()
data_test['FullDescription'] = data_test['FullDescription'].str.lower()

# Замените все, кроме букв и цифр, на пробелы
data_train['FullDescription'] = data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data_test['FullDescription'] = data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

# Примените TfidfVectorizer для преобразования текстов в векторы признаков
vectorizer = TfidfVectorizer(min_df=5) 
X_train_tfid = vectorizer.fit_transform(data_train['FullDescription'])
X_test_tfid = vectorizer.transform(data_test['FullDescription'])

# Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

# Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

# Объедините все полученные признаки в одну матрицу "объекты-признаки". 
X_train = hstack([X_train_tfid, X_train_categ])
X_test = hstack([X_test_tfid, X_test_categ])

# Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized. 
clf = Ridge(alpha=1.0, random_state=241)
clf.fit(X_train, data_train['SalaryNormalized'])

# Постройте прогнозы для двух примеров из файла salary-test-mini.csv. 
result = np.round(clf.predict(X_test), 2)

# [56555.62 37188.32]
answer = '{} {}'.format(str(result[0]), str(result[1]))

file = open(PATH + 'lesson_4/1.txt', 'w')
file.write(answer)


56555.62 37188.32
